In [297]:
import pandas as pd
import csv

In [298]:
csvPath = "../Resources/video_game.csv"

In [299]:
game_df = pd.read_csv(csvPath)
game_df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [300]:
totalPlayers = game_df["SN"].nunique()
totalPlayers

576

In [301]:
#Purchasing Analysis

uniqueItems = game_df["Item Name"].nunique()
print(f"Number of unique items: {uniqueItems}")
avgPrice = round(game_df["Price"].mean(),2)
print(f"Average purchase price: ${avgPrice}")
numItems = game_df["Item Name"].count()
print(f"Number of Purchases: {numItems}")
revenue = game_df["Price"].sum()
print(f"Total revenue: ${revenue}")

pd.DataFrame([{"Number of Unique Items": uniqueItems,
              "Average Price": avgPrice, 
               "Number of Purchases": numItems,
               "Total Revenue": revenue,}])

Number of unique items: 179
Average purchase price: $3.05
Number of Purchases: 780
Total revenue: $2379.77


,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,3.05,780,2379.77


In [302]:
#Gender Demographics

all_users = []
unique_users = []
genders =[]
ages = []

with open(csvPath) as csvFile:
    csvReader = csv.reader(csvFile,delimiter=",")
    #unique_SN_list = game_df["SN"].unique()
    header = next(csvReader,None)
    for row in csvReader:
        SN = row[1]
        gender = row[3]
        age = row[2]
        all_users.append(SN)
        if SN not in unique_users:
            unique_users.append(SN)
            genders.append(gender)
            ages.append(age)

maleCount = genders.count("Male")
malePerc = round(maleCount/totalPlayers*100,2)
femaleCount = genders.count("Female")
femalePerc = round(femaleCount/totalPlayers*100,2)
otherCount = genders.count("Other / Non-Disclosed")
otherPerc = round(otherCount/totalPlayers*100,2)
            
gender_dicts = [{"Total Count": maleCount, "Gender": "Male", "Percentage of Players":malePerc},
               {"Total Count": femaleCount, "Gender": "Female", "Percentage of Players":femalePerc},
               {"Total Count": otherCount, "Gender": "Other / Non-Disclosed", "Percentage of Players":otherPerc}]


gender_df = pd.DataFrame(gender_dicts)
gender_df = gender_df.set_index("Gender")
gender_df

,Total Count,Percentage of Players
Gender,,
Male,484,84.03
Female,81,14.06
Other / Non-Disclosed,11,1.91


In [303]:
unique_users_list=[]

for i in range(len(unique_users)):
    SN = unique_users[i]
    age = int(ages[i])
    unique_users_dict = {"SN":SN , "Age":age}
    unique_users_list.append(unique_users_dict)
    
unique_user_df = pd.DataFrame(unique_users_list)
unique_user_df

,SN,Age
0,Lisim78,20
1,Lisovynya38,40
2,Ithergue48,24
3,Chamassasya86,24
4,Iskosia90,23
...,...,...
571,Hala31,21
572,Jiskjask80,11
573,Aethedru70,21
574,Yathecal72,20


In [304]:
malePurchases_df = game_df.loc[game_df["Gender"]=="Male",:]
malePurchases = malePurchases_df["Item Name"].count()
avgMalePrice = round(malePurchases_df["Price"].mean(),2)
totalMalePrice = round(malePurchases_df["Price"].sum(),2)
avgPricePerMale = round(totalMalePrice/maleCount,2)

femalePurchases_df = game_df.loc[game_df["Gender"]=="Female",:]
femalePurchases = femalePurchases_df["Item Name"].count()
avgFemalePrice = round(femalePurchases_df["Price"].mean(),2)
totalFemalePrice = round(femalePurchases_df["Price"].sum(),2)
avgPricePerFemale = round(totalFemalePrice/femaleCount,2)

otherPurchases_df = game_df.loc[game_df["Gender"]=="Other / Non-Disclosed",:]
otherPurchases = otherPurchases_df["Item Name"].count()
avgOtherPrice = round(otherPurchases_df["Price"].mean(),2)
totalOtherPrice = round(otherPurchases_df["Price"].sum(),2)
avgPricePerOther = round(totalOtherPrice/otherCount,2)

# print(f"Purchase Count: {malePurchases}")
# print(f"Average Purchase Price: {avgMalePrice}")
# print(f"Total Purchase Value: {totalMalePrice}")
# print(f"Average Purchase per Person: {avgPricePerMale}")

gender_purchase_dicts = [{"Purchase Count": malePurchases, "Gender": "Male", "Average Purchase Price":avgMalePrice, "Total Purchase Value":totalMalePrice, "Avg Total Purchase per Person":avgPricePerMale},
                        {"Purchase Count": femalePurchases, "Gender": "Female", "Average Purchase Price":avgFemalePrice, "Total Purchase Value":totalFemalePrice, "Avg Total Purchase per Person":avgPricePerFemale},
                        {"Purchase Count": otherPurchases, "Gender": "Other / Non-Disclosed", "Average Purchase Price":avgOtherPrice, "Total Purchase Value":totalOtherPrice, "Avg Total Purchase per Person":avgPricePerOther}]
gender_purchase_df = pd.DataFrame(gender_purchase_dicts)
gender_purchase_df = gender_purchase_df.set_index("Gender")
gender_purchase_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Male,652,3.02,1967.64,4.07
Female,113,3.20,361.94,4.47
Other / Non-Disclosed,15,3.35,50.19,4.56


In [305]:
game_df["Age"].max()

45

In [314]:
#Age Demographics
percent_list = []
age_count = []


bins = [0,9.9,14.9,19.9,24.9,29.9,34.9,39.9,50]
age_ranges = ["<10","10-14","15-19","20-24","25-29","30-34","35-39","40+"]

unique_user_df["Age Ranges"] = pd.cut(unique_user_df["Age"], bins, labels=age_ranges)
unique_user_df = unique_user_df.drop("Age",1)


playerPerc = round(1/576*100,2)
for i in range(576):
    percent_list.append(playerPerc)
    age_count.append(1)

unique_user_df["Percentage of Players"] = percent_list
unique_user_df["Player Count"] = age_count
#unique_user_df.reset_index()

grouped_age_df = unique_user_df.groupby("Age Ranges")
grouped_age_df.sum()

,Percentage of Players,Player Count
Age Ranges,,
<10,2.89,17
10-14,3.74,22
15-19,18.19,107
20-24,43.86,258
25-29,13.09,77
30-34,8.84,52
35-39,5.27,31
40+,2.04,12
